In [0]:
!pip install protobuf pillow lxml Cython jupyter matplotlib pandas opencv-python
!apt-get install -y -qq protobuf-compiler python-pil python-lxml

In [0]:
!git clone https://github.com/tensorflow/models

In [0]:
!git clone https://github.com/cocodataset/cocoapi.git

In [0]:
%cd cocoapi/PythonAPI
!make 

In [0]:
!cp -r pycocotools /content/models/research/

In [0]:
%cd /content/models/research/
!protoc object_detection/protos/*.proto --python_out=.

In [0]:
import os
os.environ['PYTHONPATH'] = "/content/models/research:/content/models/research/slim"

In [0]:
!python object_detection/builders/model_builder_test.py

In [0]:
%cd /content
!rm -rf utn_tf
!git clone https://gitlab.com/arkanwe/utn_tf

In [0]:
%cd /content/utn_tf/samples
!mkdir train

In [0]:
!python scripts/gen_set.py raw train 100

In [0]:
!python scripts/xml2csv.py train

In [0]:
!python scripts/xml2csv.py eval

In [0]:
!python scripts/gen_tfrec.py --csv_input=train/train.csv --image_dir=train --output_path=train/train.record

In [0]:
!python scripts/gen_tfrec.py --csv_input=eval/eval.csv --image_dir=eval --output_path=eval/eval.record

In [0]:
%cd /content/
!curl http://download.tensorflow.org/models/object_detection/ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz -o ssdlite_mobilenet_v2.tar.gz

In [0]:
!gunzip ssdlite_mobilenet_v2.tar.gz
!tar -xvf  ssdlite_mobilenet_v2.tar

In [0]:
%cd /content/models/research
get_ipython().system_raw('python3 object_detection/legacy/train.py \
    --logtostderr \
    --pipeline_config_path=/content/utn_tf/training/model.config \
    --train_dir=/content/utn_tf/training &')

In [0]:
%cd /content/models/research
get_ipython().system_raw('python object_detection/legacy/eval.py \
    --logtostderr \
    --pipeline_config_path=/content/utn_tf/training/model.config \
    --checkpoint_dir=/content/utn_tf/training \
    --eval_dir=/content/utn_tf/training &')

In [0]:
%cd /content/utn_tf 
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

In [0]:
get_ipython().system_raw('tensorboard --logdir /content/utn_tf/training --host 0.0.0.0 --port 6006 &')

In [0]:
get_ipython().system_raw('./content/utn_tf/ngrok http 6006 &')

In [0]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [0]:
!ls /content/utn_tf/training

In [0]:
!mkdir /content/utn_tf/training/inf_graph
!python object_detection/export_inference_graph.py \
    --input_type image_tensor \
    --pipeline_config_path /content/utn_tf/training/model.config \
    --trained_checkpoint_prefix /content/utn_tf/training/model.ckpt-385 \
    --output_directory /content/utn_tf/training/inf_graph

In [0]:
!zip -r /content/utn_tf/training/inf_graph.zip /content/utn_tf/training/inf_graph

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
uploaded = drive.CreateFile({'title': 'inf_graph.zip'})
uploaded.SetContentFile('/content/utn_tf/training/inf_graph.zip')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

In [0]:
python detectar_img.py training/inf_graph/frozen_inference_graph.pb samples/label_map.txt ./test